In [1]:
import numpy as np

from core.Convolutional import Convolutional
from core.Flatten import Flatten
from core.Softmax import Softmax
from core.ReLU import ActivationLayer
from core.MaxPool import MaxPool
from core.FullyConnected import FullyConnected
from core.Model import Model

from training.train import train, test_model

from torchvision import datasets
from torch.utils.data import DataLoader

In [2]:
class ToNumpy:
    def __call__(self, pic):
        return np.array(pic, dtype=np.float32)  # convert to float32 NumPy array
mnist_train = datasets.MNIST(root='./training/data', train=True, download=True, transform=ToNumpy())
train_loader = DataLoader(mnist_train, batch_size=32, shuffle=True, num_workers=2)

# Train custom model or alteratively load a pretrained one

In [ ]:
layers = [
    Convolutional(1,16,3,1,1),
    ActivationLayer(),
    MaxPool(16,2,2),
    Convolutional(16,32,3,1,1),
    ActivationLayer(),
    MaxPool(32,2,2),
    Flatten(),
    FullyConnected(1568,128),
    ActivationLayer(),
    FullyConnected(128, 10),
    Softmax(temperature=1),
]
model = Model(layers)

In [ ]:
train(model, train_loader, 3, 0.001)

# Or load a pretrained model

In [14]:
layers_data = np.load("./saved_models/layers.npz", allow_pickle=True)
layers_loaded = [layers_data[f"layer_{i}"] for i in range(len(layers_data.files))]
params = np.load("./saved_models/model_data.npz", allow_pickle=True)
model = Model(layers_loaded)
for key in params.files:
    setattr(model, key, params[key])


# Test the model

In [13]:
test_dataset = mnist_train = datasets.MNIST(root='./training/data', train=False, download=True, transform=ToNumpy())
print(test_model(model,test_dataset,32))

0.9435
